In [1]:
%load_ext autoreload

In [2]:
autoreload 2

In [3]:
# import sys
# !{sys.executable} -m pip install -r ../../requirements.txt

  Obtaining dependency information for contourpy==1.3.0 from https://files.pythonhosted.org/packages/a9/97/3f89bba79ff6ff2b07a3cbc40aa693c360d5efa90d66e914f0ff03b95ec7/contourpy-1.3.0-cp310-cp310-win_amd64.whl.metadata
  Using cached contourpy-1.3.0-cp310-cp310-win_amd64.whl.metadata (5.4 kB)
  Obtaining dependency information for cycler==0.12.1 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Obtaining dependency information for fonttools==4.55.0 from https://files.pythonhosted.org/packages/3e/67/93939482715e629c4bd6fd1f3d154c1bf45b81ee383f50e44d31fa542f83/fonttools-4.55.0-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/167.9 kB ? eta -:--:--
     -------------- ------------------------ 61.4/167.9 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 167.9/167.9 kB 2.0

ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\rocke\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\~il\\_imaging.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

helper_path = os.path.abspath(os.path.join('../helper'))
if helper_path not in sys.path:
    sys.path.append(helper_path)

from helper.dataclass import HDDDataset
from helper.models.mymodel import MyModel
from helper.preprocessing import *
from helper.metrics import *
from helper.eda import *
from sklearn.model_selection import train_test_split
from helper.metrics import *
from helper.saver import Saver

## Data preprocessing

In [5]:
hdd_dataset = HDDDataset.read_csv('ST14000NM001G.csv')

In [6]:
preprocessor = Preprocessing(hdd_dataset)
preprocessor.clear_unused_data()
preprocessor.add_target_column()
preprocessor.prepare_train_df()
preprocessor.add_time_features()
preprocessor.train_test_val_split(train_size=0.8, test_size=0.2, val_size=0)
preprocessor.normalize_data(method='boxcoxlog', inplace=True)

Clearing unused columns...
Adding target column...
Prepairing train dataset...
Adding time features to unsplitted dataset...
Splitting train dataset...
Normalizing splitted dataset with boxcoxlog...


## Training

In [7]:
model = MyModel('LightGBM')
model.fit(preprocessor.X_train, preprocessor.y_train, method='grid', epochs=20)
print(model.get_hyperparams())

=== Оптимизация гиперпараметров для LightGBM... ===
Лучшие параметры для LightGBM:
{'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 200}
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.01, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 200, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbose': -1}


## Testing

In [8]:
preds = model.predict(preprocessor.X_test)
metrics = Metrics.get_metrics(preds, preprocessor.y_test)
print(metrics)

{'accuracy': 0.9995512383976001, 'precision': 0.7912087912087912, 'recall': 0.490272373540856, 'f1': 0.6054054054054054, 'auc_roc': 0.7450907322560429}


In [9]:
from sklearn.metrics import classification_report
print(classification_report(preprocessor.y_test, preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1463001
           1       0.79      0.49      0.61      1028

    accuracy                           1.00   1464029
   macro avg       0.90      0.75      0.80   1464029
weighted avg       1.00      1.00      1.00   1464029



## Logging

In [9]:
Saver.save(model, preprocessor, metrics, save_csv=False)